
# Week 7 — Serve an ML Model with Flask (VS Code / JupyterLab Friendly)

This notebook creates a minimal Flask web app to serve a trained scikit-learn model.
No `%%writefile` magic is used — files are created with regular Python `open(..., "w")` calls.

**What you’ll build**
- `model.pkl` — trained Iris classifier pipeline
- `schema.json` — feature names/order + class names
- `app.py` — Flask server with `/` and `/predict`
- `templates/index.html` — simple HTML form
- `requirements.txt`, `Procfile`, `README.md` — deployment & docs

> Run each cell top-to-bottom. The last cell launches the server.


## 1) Create a project folder and enter it

In [1]:

# Purpose: make a clean workspace for the Flask app and move into it
import os, pathlib
proj = pathlib.Path("Week7_FlaskApp")
proj.mkdir(exist_ok=True)
os.chdir(proj)
print("Working in:", os.getcwd())


Working in: /Users/vicev/Downloads/Week7_FlaskApp


## 2) Install dependencies

In [2]:

# Purpose: install everything needed to train + serve the model
# (If these are already installed, pip will skip reinstallation.)
!pip install -q flask joblib scikit-learn numpy


## 3) Train a model (Iris) and save it + schema

In [3]:

# Purpose: train a small model pipeline and save:
#   - model.pkl (for Flask to load)
#   - schema.json (feature names/order + class names for the UI)

from sklearn.datasets import load_iris
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import joblib, json

# Load dataset
iris = load_iris()
X, y = iris.data, iris.target
feature_names = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
class_names = iris.target_names.tolist()  # ['setosa','versicolor','virginica']

# Simple pipeline
pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("lr", LogisticRegression(max_iter=1000))
])
pipe.fit(X, y)

# Save model
joblib.dump(pipe, "model.pkl")

# Save schema describing the features the UI should show (order matters!)
schema = {
    "features": [
        {"name": "sepal_length", "label": "Sepal Length (cm)"},
        {"name": "sepal_width",  "label": "Sepal Width (cm)"},
        {"name": "petal_length", "label": "Petal Length (cm)"},
        {"name": "petal_width",  "label": "Petal Width (cm)"}
    ],
    "class_names": class_names
}
with open("schema.json", "w") as f:
    json.dump(schema, f, indent=2)

print("✅ Saved model.pkl and schema.json")


✅ Saved model.pkl and schema.json


## 4) Create the Flask app (`app.py`)

In [4]:

# Purpose: write app.py using standard Python file I/O (works in VS Code / JupyterLab)
app_py = """from flask import Flask, render_template, request
import joblib, json
import numpy as np

app = Flask(__name__)

# Load model and schema once at startup
model = joblib.load("model.pkl")
with open("schema.json") as f:
    schema = json.load(f)
FEATURES = [f["name"] for f in schema["features"]]
CLASS_NAMES = schema["class_names"]

@app.route("/", methods=["GET"])
def home():
    # Pass schema to template so it can render fields dynamically
    return render_template("index.html", features=schema["features"], prediction=None, values={})

@app.route("/predict", methods=["POST"])
def predict():
    try:
        # Read features in the exact order specified by schema
        values = {}
        row = []
        for f in FEATURES:
            raw = request.form.get(f, "").strip()
            values[f] = raw
            row.append(float(raw))
        pred_idx = model.predict([row])[0]
        pred_label = CLASS_NAMES[int(pred_idx)]
        return render_template("index.html", features=schema["features"], prediction=f"Prediction: {pred_label}", values=values)
    except Exception as e:
        return render_template("index.html", features=schema["features"], prediction=f"Error: {e}", values=request.form)

if __name__ == "__main__":
    # For local development
    app.run(debug=True)
"""
with open("app.py", "w", encoding="utf-8") as f:
    f.write(app_py)

print("✅ Wrote app.py")


✅ Wrote app.py


## 5) Create the HTML template (`templates/index.html`)

In [5]:

# Purpose: simple UI with number inputs bound to the schema features
# Tip: Jinja loop renders inputs for any features defined in schema.json
import os, pathlib
pathlib.Path("templates").mkdir(exist_ok=True)

html = """<!DOCTYPE html>
<html>
<head>
  <meta charset="utf-8"/>
  <title>ML Predictor</title>
  <style>
    body { font-family: Arial, sans-serif; max-width: 720px; margin: 48px auto; }
    h1 { margin-bottom: 8px; }
    .card { padding: 20px; border: 1px solid #eee; border-radius: 12px; box-shadow: 0 2px 10px rgba(0,0,0,0.04); }
    .row { display: grid; grid-template-columns: 1fr 1fr; gap: 12px; margin: 12px 0; }
    label { display: block; font-size: 14px; margin-bottom: 4px; }
    input { width: 100%; padding: 10px; border-radius: 8px; border: 1px solid #ccc; }
    button { padding: 10px 16px; border: none; border-radius: 8px; cursor: pointer; }
    .primary { background: #111; color: #fff; }
    .result { margin-top: 16px; font-weight: bold; }
  </style>
</head>
<body>
  <h1>🧠 Machine Learning Predictor</h1>
  <p>Enter feature values and get a prediction.</p>
  <div class="card">
    <form action="/predict" method="post">
      <div class="row">
        {% for f in features %}
          <div>
            <label for="{{ f.name }}">{{ f.label }}</label>
            <input type="number" step="any" name="{{ f.name }}" id="{{ f.name }}" placeholder="{{ f.label }}"
                   value="{{ values.get(f.name, '') }}" required>
          </div>
        {% endfor %}
      </div>
      <button class="primary" type="submit">Predict</button>
    </form>
    {% if prediction %}
      <div class="result">{{ prediction }}</div>
    {% endif %}
  </div>
</body>
</html>
"""
with open("templates/index.html", "w", encoding="utf-8") as f:
    f.write(html)

print("✅ Wrote templates/index.html")


✅ Wrote templates/index.html


## 7) (Optional) Write a README with quickstart

In [7]:

# Purpose: doc for your repo + quick run steps
readme = """# Week 7 — Serve an ML Model with Flask

This demo trains a small Iris classifier and serves it via Flask.

## Quickstart (Local)
```bash
pip install -r requirements.txt
python app.py
# open http://127.0.0.1:5000/
```

## Project Layout
```
Week7_FlaskApp/
├─ app.py
├─ model.pkl
├─ schema.json
├─ requirements.txt
└─ templates/
   └─ index.html
```

## How it Works
- `model.pkl`: scikit-learn pipeline (StandardScaler + LogisticRegression)
- `schema.json`: lists feature names/order + class names for display
- `index.html`: renders inputs dynamically from schema
- `app.py`: loads model & schema, predicts from form inputs
"""
with open("README.md", "w") as f:
    f.write(readme)
print("✅ Wrote README.md")


✅ Wrote README.md


## 8) Run the Flask app locally

In [8]:

# Purpose: launch the web server locally
# NOTE: This blocks the notebook while the server runs.
# Stop it with the stop button or interrupt kernel when you're done.
!python3 app.py


 * Serving Flask app 'app'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
 * Debugger is active!
 * Debugger PIN: 108-044-395
127.0.0.1 - - [07/Oct/2025 23:54:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [07/Oct/2025 23:54:04] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [07/Oct/2025 23:54:15] "POST /predict HTTP/1.1" 200 -
^C



## (Optional) Swap in your own model (e.g., Titanic)

Replace the training cell with your own code that saves a compatible `model.pkl` and writes a matching `schema.json` (feature names, order, class names).


In [ ]:

# Example schema writer for a Titanic model (run after you train & save your Titanic model):
# import joblib, json
# joblib.dump(your_pipeline_or_model, "model.pkl")
#
# schema = {
#   "features": [
#     {"name": "Pclass", "label": "Passenger Class"},
#     {"name": "Sex", "label": "Sex (0=female,1=male)"},
#     {"name": "Age", "label": "Age"},
#     {"name": "Fare", "label": "Fare"}
#     # ...add all features your model expects, in order
#   ],
#   "class_names": ["not_survived","survived"]
# }
# with open("schema.json","w") as f:
#     json.dump(schema, f, indent=2)
#
# After updating schema/model, you can rerun the app without changing app.py or the template.
print("This cell is an example snippet. Customize for your own model if needed.")
